In [2]:
import random
import tkinter as tk

HEIGHT = 600
WIDTH = 700
color = ["lemonchiffon","plum"]
cpt = 0
coord_red_ord = []
coord_red_abs = []
coord_yellow_ord = []
coord_yellow_abs = []
coord_red_tot = []
coord_yellow_tot = []




nom_joueur1 = input("Quel est ton nom joueur 1 ?") # identification du joueur 1
nom_joueur2 = input("Quel est ton nom joueur 2 ?") # identification du joueur 2

def affichage():
    """Permet d'afficher le nom du joueur qui doit jouer"""
    global tour_joueur
    tour_joueur += 1
    if tour_joueur == len(liste_joueurs):
        tour_joueur = 0
    label3.config(text="C'est au tour de " + liste_joueurs[tour_joueur] + " de jouer")


liste_joueurs = [nom_joueur1, nom_joueur2]

tour_joueur = random.randint(0, 1)

list1 = [((0,0),(100,100)),((0,100),(100,200)),((0,200),(100,300)),
         ((0,300),(100,400)),((0,400),(100,500)),((0,500),(100,600))]

list2 = [((100,0),(200,100)),((100,100),(200,200)),((100,200),(200,300)),
         ((100,300),(200,400)),((100,400),(200,500)),((100,500),(200,600))]

list3 =  [((200,0),(300,100)),((200,100),(300,200)),((200,200),(300,300)),
         ((200,300),(300,400)),((200,400),(300,500)),((200,500),(300,600))]

list4 =  [((300,0),(400,100)),((300,100),(400,200)),((300,200),(400,300)),
         ((300,300),(400,400)),((300,400),(400,500)),((300,500),(400,600))]

list5 = [((400,0),(500,100)),((400,100),(500,200)),((400,200),(500,300)),
         ((400,300),(500,400)),((400,400),(500,500)),((400,500),(500,600))]

list6 =  [((500,0),(600,100)),((500,100),(600,200)),((500,200),(600,300)),
         ((500,300),(600,400)),((500,400),(600,500)),((500,500),(600,600))]

list7 =  [((600,0),(700,100)),((600,100),(700,200)),((600,200),(700,300)),
         ((600,300),(700,400)),((600,400),(700,500)),((600,500),(700,600))]

#  Création des cercles


def jeton(list):
    "Permet de placer un jeton dans la première colonne et d'afficher le nom du joueur qui doit jouer son tour"
    global cpt, coord_red_abs, coord_yellow_abs, coord_red_ord, coord_yellow_ord, coord_red_tot, coord_yellow_tot
    cpt = 1 - cpt
    canvas.create_oval(list[-1],fill = color[cpt])
    if cpt == 0:
        coord_yellow_abs.append(list[-1][1][0])
        coord_yellow_ord.append(list[-1][1][1])
        coord_yellow_tot.append(list[-1][1])
    else:
        coord_red_abs.append(list[-1][1][0])
        coord_red_ord.append(list[-1][1][1])
        coord_red_tot.append(list[-1][1])
    del list[-1]
    if win_try(coord_yellow_abs,coord_yellow_ord) or win_try(coord_yellow_ord,coord_yellow_abs) or \
        win_try(coord_red_abs,coord_red_ord) or win_try(coord_red_ord,coord_red_abs) or \
        diag(coord_red_tot) or diag(coord_yellow_tot) or diag2(coord_red_tot) or diag2(coord_yellow_tot):
        print("Partie terminée")
        racine.destroy()
    affichage()

# supprimer le dernier jeton jouer

def retour(event):
    global cpt, coord_red_abs, coord_yellow_abs, coord_red_ord, coord_yellow_ord, coord_red_tot, coord_yellow_tot
    try:
        if cpt == 1:
            couple = coord_red_tot[-1]
            del coord_red_tot[-1]
            del coord_red_ord[-1]
            del coord_red_abs[-1]
        else:
            couple = coord_yellow_tot[-1]
            del coord_yellow_tot[-1]
            del coord_yellow_ord[-1]
            del coord_yellow_abs[-1]
        if couple[0] == 100:
            list1.append(((0,couple[1]-100),(100,couple[1])))
            jeton = canvas.find_closest(50,couple[1]-50)
        if couple[0] == 200:
            list2.append(((100,couple[1]-100),(200,couple[1])))
            jeton = canvas.find_closest(150,couple[1]-50)
        if couple[0] == 300:
            list3.append(((200,couple[1]-100),(300,couple[1])))
            jeton = canvas.find_closest(250,couple[1]-50)
        if couple[0] == 400:
            list4.append(((300,couple[1]-100),(400,couple[1])))
            jeton = canvas.find_closest(350,couple[1]-50)
        if couple[0] == 500:
            list5.append(((400,couple[1]-100),(500,couple[1])))
            jeton = canvas.find_closest(450,couple[1]-50)
        if couple[0] == 600:
            list6.append(((500,couple[1]-100),(600,couple[1])))
            jeton = canvas.find_closest(550,couple[1]-50)
        if couple[0] == 700:
            list7.append(((700,couple[1]-100),(700,couple[1])))
            jeton = canvas.find_closest(650,couple[1]-50)
        canvas.delete(jeton)
        cpt = 1 - cpt
    except:
        pass





racine = tk.Tk() # Création de la fenêtre racine
racine.title("JEU PUISSANCE 4") # ajoute un titre

label = tk.Label(racine, text="Prêt à vous affronter au Puissance 4 ?", font=("helvetica", "20")) # création du widget
label1 = tk.Label(racine, text="Joueur 1 = " + nom_joueur1, font=("helvetica", "30")) # création du joeur 1
label2 = tk.Label(racine, text="Joueur 2 = " + nom_joueur2, font=("helvetica", "30")) # création du joeur 2
label3 = tk.Label(racine, text="C'est au tour de " + liste_joueurs[tour_joueur] + " de jouer", font=("helvetica", "10"))
label.grid(row=0,columnspan=7) # positionnement du widget
label1.grid(row=1,columnspan = 7) # positionnement du nom du joueur 1
label2.grid(row=2,columnspan = 7) # positionnement du nom du joeur 2
label3.grid(row=3,columnspan = 7)

button1 = tk.Button(racine,command=lambda : jeton(list1))
button1.grid(row = 4, column = 0)
button2 = tk.Button(racine,command=lambda : jeton(list2))
button2.grid(row = 4, column = 1)
button3 = tk.Button(racine,command=lambda : jeton(list3))
button3.grid(row = 4, column = 2)
button4 = tk.Button(racine,command=lambda : jeton(list4))
button4.grid(row = 4, column = 3)
button5 = tk.Button(racine,command=lambda : jeton(list5))
button5.grid(row = 4, column = 4)
button6 = tk.Button(racine,command=lambda : jeton(list6))
button6.grid(row = 4, column = 5)
button7 = tk.Button(racine,command=lambda : jeton(list7))
button7.grid(row = 4, column = 6)
button8 = tk.Button(racine,text="rewind",bg="plum",command = retour)
button8.grid(row = 4, column = 7)

canvas = tk.Canvas(racine, bg="royalblue", height = HEIGHT, width = WIDTH,cursor = "pirate")
canvas.grid(row = 5, columnspan = 7)

#  Création des lignes

for i in range(7):
    for j in range(8):
        canvas.create_rectangle((i * 100,j * 100),((i+1) *  100,(j+1) * 100),fill = "royalblue")  #changer en ligne


# Condition de victoire

def win_try(list1,list2):
    l = len(list1)
    if l > 3:
        for i in range(l):
            k = list1.count(list1[i])
            if k > 3:
                indices_i = [j for j in range(l) if list1[j]==list1[i]]
                ord = [list2[l] for l in indices_i]
                ord.sort()
                if k == 4:
                    if ord[0] + ord[1] + ord[2] + ord[3] == ord[0]*4 + 600:
                        return True
                elif k == 5:
                    if ord[4] + ord[1] + ord[2] + ord[3] == ord[1]*4 + 600:
                        return True
                elif k == 6:
                    if ord[4] + ord[5] + ord[2] + ord[3] == ord[2]*4 + 600:
                        return True
                elif k == 7:
                    if ord[4] + ord[5] + ord[6] + ord[3] == ord[3]*4 + 600:
                        return True


def diag(list):    #diagonale droite bas ou haut gauche
    l = len(list)
    for (abs,ord) in list:
        if (abs+100,ord+100) in list:
            if (abs+200,ord+200) in list:
                if (abs+300,ord+300) in list:
                    return True

def diag2(list): #diag haut droite ou bas gauche
    l = len(list)
    for (abs,ord) in list:
        if (abs-100,ord+100) in list:
            if (abs-200,ord+200) in list:
                if (abs-300,ord+300) in list:
                    return True






racine.bind('<a>',lambda event :jeton(list1))
racine.bind('<z>',lambda event :jeton(list2))
racine.bind('<e>',lambda event :jeton(list3))
racine.bind('<r>',lambda event :jeton(list4))
racine.bind('<t>',lambda event :jeton(list5))
racine.bind('<y>',lambda event :jeton(list6))
racine.bind('<u>',lambda event :jeton(list7))
racine.bind('<BackSpace>',retour)



racine.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\raagh\miniconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
TypeError: retour() missing 1 required positional argument: 'event'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\raagh\miniconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
TypeError: retour() missing 1 required positional argument: 'event'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\raagh\miniconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
TypeError: retour() missing 1 required positional argument: 'event'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\raagh\miniconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
TypeError: retour() missing 1 required positional argument: 'event'
Exception in Tkinter cal

In [48]:
import tkinter as tk
import random as random

colonne = int(input("Choisir un nombre de colonnes"))
ligne = int(input("Choisir un nombre de lignes"))
puissance = int(input("Choisir le nombre de jetons à aligner pour gagner"))
manche = int(input("Combien de manches faut-il gagner ?"))
joueur = [0,0]
color = ["lemonchiffon","plum"]


HEIGHT = ligne * 100
WIDTH = colonne * 100

racine = tk.Tk() # Création de la fenêtre racine
racine.title("JEU PUISSANCE 4") # ajoute un titre
canvas = tk.Canvas(racine, bg="royalblue", height = HEIGHT, width = WIDTH)
canvas.grid(column = 0, rowspan = 3)
label = tk.Label(racine, text="Nombre de manches à gagner : "+str(manche), font=("helvetica", "20")) 
label.grid(column = 1,row = 0)
label2 = tk.Label(racine, text="joueur 1 a gagné " + str(joueur[0]) + " manche ", font=("helvetica", "20"),bg = color[0])
label3 = tk.Label(racine, text="joueur 2 a gagné " + str(joueur[1]) + " manche ", font=("helvetica", "20"),bg = color[1])
label2.grid(column = 1,row = 1)
label3.grid(column = 1,row = 2)

cpt = random.randint(0,1)
coord = [[(i*100,j*100) for j in range(1,ligne +1)] for i in range(1,colonne+1)]
coord_yellow_abs = []
coord_yellow_ord = []
coord_red_abs = []
coord_red_ord = []
coord_red_tot = []
coord_yellow_tot = []

# Création des lignes
for i in range(colonne):
    for j in range(ligne):
        canvas.create_rectangle((i * 100,j * 100),((i+1) *  100,(j+1) * 100),fill = "royalblue")


# Placement des jetons
def jeton(event):
    global cpt, coord, coord_red_abs,coord_red_ord,coord_yellow_abs,coord_yellow_ord, coord_red_tot,coord_yellow_tot, joueur
    colonne = event.x // 100
    canvas.create_oval((coord[colonne][-1][0]-100,coord[colonne][-1][1]-100),coord[colonne][-1], fill = color[cpt])
    if cpt == 0:
        coord_yellow_abs.append(coord[colonne][-1][0])
        coord_yellow_ord.append(coord[colonne][-1][1])
        coord_yellow_tot.append(coord[colonne][-1])
    else:
        coord_red_abs.append(coord[colonne][-1][0])
        coord_red_ord.append(coord[colonne][-1][1])
        coord_red_tot.append(coord[colonne][-1])
    del coord[colonne][-1]
    if win_try(coord_yellow_abs,coord_yellow_ord) or win_try(coord_yellow_ord,coord_yellow_abs) or \
        win_try(coord_red_abs,coord_red_ord) or win_try(coord_red_ord,coord_red_abs) or \
        diag(coord_red_tot) or diag(coord_yellow_tot) or diag2(coord_red_tot) or diag2(coord_yellow_tot):
        joueur[cpt] += 1
        update()
        if joueur[cpt] == manche:
            racine.destroy()
        else:
            reboot()
    cpt = 1 - cpt


# Conditions de victoire:

def win_try(list1,list2):
    global puissance
    L = []
    if len(list1) > puissance - 1:
        for i in range(len(list1)):
            var1_i = list1.count(list1[i])
            if var1_i > puissance - 1:
                indices_var1_i = [j for j in range(len(list1)) if list1[j]==list1[i]]
                var2 = [list2[l] for l in indices_var1_i]
                var2.sort()
                for k in range(0,len(var2)-puissance+1):
                    S = 0
                    for i in range(k,k+puissance):
                        S += var2[i]
                    if S == var2[k]*puissance +(puissance)*(puissance-1)//2 * 100:
                        return True


def diag(list):    #diagonale droite bas ou haut gauche
    global puissance
    for (abs,ord) in list:
        S = 0
        for i in range(puissance):
            if (abs+100,ord+100) in list:
                S += 1
                abs += 100
                ord += 100
        if S == puissance - 1:
            return True
                
def diag2(list):    #diag haut droite ou bas gauche
    global puissance
    for (abs,ord) in list:
        S = 0
        for i in range(puissance):
            if (abs-100,ord+100) in list:
                S += 1
                abs -= 100
                ord += 100
        if S == puissance - 1:
            return True

def retour(event):
    global cpt, coord_red_abs, coord_yellow_abs, coord_red_ord, coord_yellow_ord, coord_red_tot, coord_yellow_tot
    try:
        if cpt == 0:
            couple = coord_red_tot[-1]
            del coord_red_tot[-1]
            del coord_red_ord[-1]
            del coord_red_abs[-1]
        else:
            couple = coord_yellow_tot[-1]
            del coord_yellow_tot[-1]
            del coord_yellow_ord[-1]
            del coord_yellow_abs[-1]
        for i in range(colonne+1):
            if couple[0] == 100*(i+1):
                coord[i].append(couple)
                jeton = canvas.find_closest(couple[0]-50,couple[1]-50)
        canvas.delete(jeton)
        cpt = 1 - cpt
    except:
        pass



def reboot():
    global cpt, coord, coord_red_abs,coord_red_ord,coord_yellow_abs,coord_yellow_ord, coord_red_tot,coord_yellow_tot
    coord = [[(i*100,j*100) for j in range(1,ligne +1)] for i in range(1,colonne+1)]
    coord_yellow_abs = []
    coord_yellow_ord = []
    coord_red_abs = []
    coord_red_ord = []
    while len(coord_red_tot) != 0 and len(coord_yellow_tot) != 0:
        jeton = canvas.find_closest(coord_red_tot[-1][0]-50,coord_red_tot[-1][1]-50)
        canvas.delete(jeton)
        jeton2 = canvas.find_closest(coord_yellow_tot[-1][0]-50,coord_yellow_tot[-1][1]-50)
        canvas.delete(jeton2)
        del coord_red_tot[-1]
        del coord_yellow_tot[-1]
    if cpt == 0:
        jeton2 = canvas.find_closest(coord_yellow_tot[-1][0]-50,coord_yellow_tot[-1][1]-50)
        canvas.delete(jeton2)
    else:
        jeton = canvas.find_closest(coord_red_tot[-1][0]-50,coord_red_tot[-1][1]-50)
        canvas.delete(jeton)
    

def update():
    if joueur[0] > 1:
        label2["text"] = "joueur 1 a gagné " + str(joueur[0]) + " manches"
    else:
        label2["text"] = "joueur 1 a gagné " + str(joueur[0]) + " manche"
    if joueur[1] > 1:
        label3["text"] = "joueur 2 a gagné " + str(joueur[1]) + " manches"
    else:
        label3["text"] = "joueur 2 a gagné " + str(joueur[1]) + " manche"




racine.bind('<Button-1>',jeton)
racine.bind('<BackSpace>',retour)


racine.mainloop()

# changer le tour des joueurs
# taille de fenetre unique
# perfectionner l'interface graphique

In [10]:
import tkinter as tk
import random as random
import math as math

colonne = int(input("Choisir un nombre de colonnes"))
ligne = int(input("Choisir un nombre de lignes"))
puissance = int(input("Choisir le nombre de jetons à aligner pour gagner"))
manche = int(input("Combien de manches faut-il gagner ?"))
joueur = [0,0]
color = ["lemonchiffon","plum"]


HEIGHT = 700
WIDTH = 700

hauteur = math.floor(HEIGHT / ligne)
largeur = math.floor(WIDTH / colonne)

def sauvegarder():
    global cpt, coord_red_tot,coord_yellow_tot,coord
    f_input=open("puissance4.txt",'w')
    f_input.write(str(coord_red_tot) + "\n")
    f_input.write(str(coord_yellow_tot) + "\n")
    f_input.close()


def charger():
    global cpt, coord_red_tot,coord_yellow_tot, coord,coord_red_abs,coord_red_ord,coord_yellow_abs,coord_yellow_ord
    reboot()
    f_output=open("puissance4.txt",'r') #attention message d'erreurrs
    coord_red_tot = trad(list(f_output.readline()))
    coord_yellow_tot = trad(list(f_output.readline()))
    f_output.close()
    coord = [[(i*largeur,j*hauteur) for j in range(1,ligne +1)] for i in range(1,colonne+1)]
    for i in coord_red_tot:
        canvas.create_oval((i[0]-largeur,i[1]-hauteur),i, fill = "plum")
        for j in coord:
            if i in j:
                j.remove(i)
    for i in coord_yellow_tot:
        canvas.create_oval((i[0]-largeur,i[1]-hauteur),i, fill = "lemonchiffon")
        for j in coord:
            if i in j:
                j.remove(i)
    coord_red_abs = [i[0] for i in coord_red_tot]
    coord_red_ord = [i[1] for i in coord_red_tot]
    coord_yellow_abs = [i[0] for i in coord_yellow_tot]
    coord_yellow_ord = [i[1] for i in coord_yellow_tot]
    


def trad(list):
    l = []
    z = [ '[', '(',',',']',')','\n',' ']
    l = [int(k) for k in list if k not in z]
    l.append(1)
    L = []
    if len(l)%3 == 1:
        l.pop()
    for i in range(0,len(l),3):
        j = int(str(l[i])+str(l[i+1])+str(l[i+2]))
        L.append(j)
    W = [(L[i],L[i+1]) for i in range(0,len(L)-1,2)]
    return W


racine = tk.Tk() # Création de la fenêtre racine
racine.title("JEU PUISSANCE 4") # ajoute un titre
canvas = tk.Canvas(racine, bg="royalblue", height = HEIGHT, width = WIDTH)
canvas.grid(column = 0, rowspan = 5)
label = tk.Label(racine, text="Nombre de manches à gagner : "+str(manche), font=("helvetica", "20")) 
label.grid(column = 1,row = 0)
label2 = tk.Label(racine, text="joueur 1 a gagné " + str(joueur[0]) + " manche ", font=("helvetica", "20"),bg = color[0])
label3 = tk.Label(racine, text="joueur 2 a gagné " + str(joueur[1]) + " manche ", font=("helvetica", "20"),bg = color[1])
label2.grid(column = 1,row = 1)
label3.grid(column = 1,row = 2)
button = tk.Button(racine,text="Sauvegarder la partie", font=("helvetica", "20"),command = sauvegarder) #modif
button.grid(row = 3, column = 1)
button2 = tk.Button(racine,text="Charger la partie sauvegarder",font=("helvetica", "20"),command =charger)
button2.grid(row=4, column = 1)

cpt = random.randint(0,1)
coord = [[(i*largeur,j*hauteur) for j in range(1,ligne +1)] for i in range(1,colonne+1)]
coord_yellow_abs = []
coord_yellow_ord = []
coord_red_abs = []
coord_red_ord = []
coord_red_tot = []
coord_yellow_tot = []

# Création des lignes
for i in range(colonne):
    for j in range(ligne):
        canvas.create_rectangle((i * largeur,j * hauteur),((i+1) *  largeur,(j+1) * hauteur),fill = "royalblue")


# Placement des jetons
def jeton(event):
    global cpt, coord, coord_red_abs,coord_red_ord,coord_yellow_abs,coord_yellow_ord, coord_red_tot,coord_yellow_tot, joueur
    colox = math.floor(event.x / largeur)
    canvas.create_oval((coord[colox][-1][0]-largeur,coord[colox][-1][1]-hauteur),coord[colox][-1], fill = color[cpt])
    if cpt == 0:
        coord_yellow_abs.append(coord[colox][-1][0])
        coord_yellow_ord.append(coord[colox][-1][1])
        coord_yellow_tot.append(coord[colox][-1])
    else:
        coord_red_abs.append(coord[colox][-1][0])
        coord_red_ord.append(coord[colox][-1][1])
        coord_red_tot.append(coord[colox][-1])
    del coord[colox][-1]
    if win_try(coord_yellow_abs,coord_yellow_ord) or win_try(coord_yellow_ord,coord_yellow_abs) or \
        win_try(coord_red_abs,coord_red_ord) or win_try(coord_red_ord,coord_red_abs) or \
        diag(coord_red_tot) or diag(coord_yellow_tot) or diag2(coord_red_tot) or diag2(coord_yellow_tot):
        joueur[cpt] += 1
        update()
        if joueur[cpt] == manche:
            racine.destroy()
        else:
            reboot()
    cpt = 1 - cpt


# Conditions de victoire:

def win_try(list1,list2):
    global puissance
    L = []
    if len(list1) > puissance - 1:
        for i in range(len(list1)):
            var1_i = list1.count(list1[i])
            if var1_i > puissance - 1:
                indices_var1_i = [j for j in range(len(list1)) if list1[j]==list1[i]]
                var2 = [list2[l] for l in indices_var1_i]
                var2.sort()
                for k in range(0,len(var2)-puissance+1):
                    S = 0
                    for i in range(k,k+puissance):
                        S += var2[i]
                    if S == var2[k]*puissance +(puissance)*(puissance-1)//2 * largeur:
                        return True
                    elif S == var2[k]*puissance +(puissance)*(puissance-1)//2 * hauteur:
                        return True


def diag(list):    #diagonale droite bas ou haut gauche
    global puissance
    for (abs,ord) in list:
        S = 0
        for i in range(puissance):
            if (abs+largeur,ord+hauteur) in list:
                S += 1
                abs += largeur
                ord += hauteur
        if S == puissance - 1:
            return True
                
def diag2(list):    #diag haut droite ou bas gauche
    global puissance
    for (abs,ord) in list:
        S = 0
        for i in range(puissance):
            if (abs-largeur,ord+hauteur) in list:
                S += 1
                abs -= largeur
                ord += hauteur
        if S == puissance - 1:
            return True

def retour(event):
    global cpt, coord_red_abs, coord_yellow_abs, coord_red_ord, coord_yellow_ord, coord_red_tot, coord_yellow_tot
    try:
        if cpt == 0:
            couple = coord_red_tot[-1]
            del coord_red_tot[-1]
            del coord_red_ord[-1]
            del coord_red_abs[-1]
        else:
            couple = coord_yellow_tot[-1]
            del coord_yellow_tot[-1]
            del coord_yellow_ord[-1]
            del coord_yellow_abs[-1]
        for i in range(colonne+1):
            if couple[0] == 100*(i+1):
                coord[i].append(couple)
                jeton = canvas.find_closest(couple[0]-largeur//2,couple[1]-hauteur//2)
        canvas.delete(jeton)
        cpt = 1 - cpt
    except:
        pass



def reboot():
    global cpt, coord, coord_red_abs,coord_red_ord,coord_yellow_abs,coord_yellow_ord, coord_red_tot,coord_yellow_tot
    coord = [[(i*largeur,j*hauteur) for j in range(1,ligne +1)] for i in range(1,colonne+1)]
    coord_yellow_abs = []
    coord_yellow_ord = []
    coord_red_abs = []
    coord_red_ord = []
    while len(coord_red_tot) != 0 and len(coord_yellow_tot) != 0:
        jeton = canvas.find_closest(coord_red_tot[-1][0]-largeur // 2,coord_red_tot[-1][1]-hauteur// 2)
        canvas.delete(jeton)
        jeton2 = canvas.find_closest(coord_yellow_tot[-1][0]-largeur // 2,coord_yellow_tot[-1][1]-hauteur//2)
        canvas.delete(jeton2)
        del coord_red_tot[-1]
        del coord_yellow_tot[-1]


def update():
    if joueur[0] > 1:
        label2["text"] = "joueur 1 a gagné " + str(joueur[0]) + " manches"
    else:
        label2["text"] = "joueur 1 a gagné " + str(joueur[0]) + " manche"
    if joueur[1] > 1:
        label3["text"] = "joueur 2 a gagné " + str(joueur[1]) + " manches"
    else:
        label3["text"] = "joueur 2 a gagné " + str(joueur[1]) + " manche"




racine.bind('<Button-1>',jeton)
racine.bind('<BackSpace>',retour)


racine.mainloop()

# changer le tour des joueurs
# taille de fenetre unique
# perfectionner l'interface graphique
# List out of index range 

[100, 300, 500, 600, 600, 700, 300] [696, 696, 696, 580, 348, 696, 464] [200, 400, 600, 600, 600, 300, 200] [696, 696, 696, 464, 232, 580, 580]


In [47]:
t = ['[', '(', '3', '4', '8', ',', ' ', '6', '9', '6', ')', ',', ' ', '(', '5', '8', '0', ',', ' ', '6', '9', '6', ')', ']', '\n']
z = [ '[', '(',',',']',')','\n',' ']
l = [int(k) for k in t if k not in z]
L = []
for i in range(0,len(l),3):
    j = int(str(l[i])+str(l[i+1])+str(l[i+2]))
    L.append(j)
W = [(L[i],L[i+1]) for i in range(0,len(L)-1,2)]
print(L)
print(l)
print(W)

[348, 696, 580, 696]
[3, 4, 8, 6, 9, 6, 5, 8, 0, 6, 9, 6]
[(348, 696), (580, 696)]
